In [ ]:
import sys
sys.path.append('../RecSysRep/')

In [ ]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = ld.getICMall()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=7777)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
import os

ofp = "../models_temp/Similarity_Hybrid/seed7777/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 3749.2876352484},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'SLIM_all2' : {'topK': 4997, 'l1_ratio': 0.00036065147205858187, 'alpha': 0.03203200564199538, 'mw': 1.0785132020354402, 'workers': 8},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 270, "learning_rate": 0.00024353880350908526, "l2_reg": 0.003835663279264412, "dropout": 0.2398024286261501, "total_anneal_steps": 188362, "anneal_cap": 0.17528653915231018, "batch_size": 128, "encoding_size": 202, "next_layer_size_multiplier": 10, "max_n_hidden_layers": 1, "max_layer_size": 5000.0}
                         }


In [ ]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [ ]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
from Recommenders.Neural.MultVAERecommender import MultVAERecommender

In [ ]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [ ]:
recommender1 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender3 = EASE_R_Recommender(URM_train)

model_init(recommender1, 'SLIMall2', models_to_combine_best['SLIM_all2'])
model_init(recommender3, 'EASE', models_to_combine_best['EASE'])

In [ ]:
recommender2 = MultVAERecommender_OptimizerMask(URM_train)
model_init(recommender2, 'Neural', models_to_combine_best['Neural'])

In [ ]:
def test_EASE(evaluator):
    recommender = EASE_R_Recommender(URM_train)
    for l2 in range(4000, 6000, 200):
        recommender.fit(topK = None, normalize_matrix = False, l2_norm = l2)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(l2, ":", r_d.loc[10]['MAP'])
        
test_EASE(evaluator_validation)

In [ ]:
ofp = "../models_temp/Similarity_Hybrid/New/"

URM_train_temp, URM_validation_temp = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
recommender_temp = EASE_R_Recommender(URM_train_temp)

#model_init(recommender1, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender_temp, 'EASE', models_to_combine_best['EASE'])

In [ ]:
evaluator_validation_temp = EvaluatorHoldout(URM_validation_temp, cutoff_list=[10])
evaluator_validation_temp.evaluateRecommender(recommender_temp)

In [ ]:
recommender_temp.fit(topK = None, normalize_matrix = False, l2_norm = 3750)
evaluator_validation_temp.evaluateRecommender(recommender_temp)

In [ ]:
import numpy as np

def test_percentage_sim(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_a.W_sparse, recommender_b.W_sparse)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNScoresHybridTwoRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [ ]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3])

In [ ]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        w_1 = trial.suggest_uniform('w_1', 0, 1)
        w_2 = trial.suggest_uniform('w_2', 0, 1)
        w_3 = trial.suggest_uniform('w_3', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(2, w_1, w_2, w_3)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            trial.report(map_v, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender1, recommender2, recommender3, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-06 19:38:05,501] A new study created in memory with name: no-name-61afefb2-8fce-4d02-9ce1-08ab6a7c667d


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.2205785261720338
MultVAERecommender with weight beta: 0.5141681241029099
EASE_R_Recommender with weight gamma: 0.2652533497250564
Norm type: 2
EvaluatorHoldout: Processed 6000 (44.0%) in 5.15 min. Users per second: 19
EvaluatorHoldout: Processed 12000 (88.0%) in 10.22 min. Users per second: 20
EvaluatorHoldout: Processed 13643 (100.0%) in 11.49 min. Users per second: 20


[I 2022-01-06 19:49:35,483] Trial 0 finished with value: -0.24887657080961229 and parameters: {'w_1': 0.27380470307889015, 'w_2': 0.6382382410281798, 'w_3': 0.32925967873004225}. Best is trial 0 with value: -0.24887657080961229.


             MAP
cutoff          
10      0.248877
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.4668734071539088
MultVAERecommender with weight beta: 0.13207323984744798
EASE_R_Recommender with weight gamma: 0.4010533529986432
Norm type: 2
EvaluatorHoldout: Processed 5000 (36.6%) in 5.31 min. Users per second: 16
EvaluatorHoldout: Processed 10000 (73.3%) in 10.74 min. Users per second: 16
EvaluatorHoldout: Processed 13643 (100.0%) in 14.57 min. Users per second: 16


[I 2022-01-06 20:04:10,238] Trial 1 finished with value: -0.24972005528737962 and parameters: {'w_1': 0.6731883129935288, 'w_2': 0.19043740800422848, 'w_3': 0.5782818768183816}. Best is trial 1 with value: -0.24972005528737962.


...

            MAP
cutoff         
10      0.24998
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5661494450772648
MultVAERecommender with weight beta: 0.26115883333744666
EASE_R_Recommender with weight gamma: 0.1726917215852886
Norm type: 2
EvaluatorHoldout: Processed 11000 (80.6%) in 5.14 min. Users per second: 36
EvaluatorHoldout: Processed 13643 (100.0%) in 6.07 min. Users per second: 37


[I 2022-01-07 00:33:31,108] Trial 28 finished with value: -0.250265227007511 and parameters: {'w_1': 0.709571853051202, 'w_2': 0.32731809405315565, 'w_3': 0.21643964496892212}. Best is trial 23 with value: -0.2502994500348442.


             MAP
cutoff          
10      0.250265
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5154515842500431
MultVAERecommender with weight beta: 0.31003389756726313
EASE_R_Recommender with weight gamma: 0.17451451818269362
Norm type: 2
EvaluatorHoldout: Processed 6000 (44.0%) in 5.65 min. Users per second: 18
EvaluatorHoldout: Processed 13643 (100.0%) in 10.47 min. Users per second: 22


[I 2022-01-07 00:43:59,326] Trial 29 finished with value: -0.25019532081688445 and parameters: {'w_1': 0.9886250576272101, 'w_2': 0.5946383505538686, 'w_3': 0.33471509423367507}. Best is trial 23 with value: -0.2502994500348442.


             MAP
cutoff          
10      0.250195
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.44978136825683707
MultVAERecommender with weight beta: 0.5171101343027247
EASE_R_Recommender with weight gamma: 0.03310849744043826
Norm type: 2
EvaluatorHoldout: Processed 4000 (29.3%) in 6.12 min. Users per second: 11
EvaluatorHoldout: Processed 6000 (44.0%) in 11.97 min. Users per second: 8


In [ ]:
ofp = "../all_models/URM_all/"

recommender1a = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_all)
recommender2a = MultVAERecommender_OptimizerMask(URM_all)

model_init(recommender1a, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender2a, 'Neural', models_to_combine_best['Neural'])

In [ ]:
recommender_final = ItemKNNScoresHybridTwoRecommender(URM_all, recommender1a, recommender2a)
recommender_final.fit(0.517796)

In [ ]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
                                sep=",",
                                dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
    subm_res["user_id"].append(user_id)
    res = recommender_final.recommend(user_id, K)
    print("USER", user_id, "RECOMMENDED!")
    res = ' '.join(map(str, res))
    print(res)
    subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)